<a href="https://colab.research.google.com/github/s24966904/note/blob/main/autoencoder_cntk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自編碼器Auto-Encoder  (cntk)

###測試於cntk 2.6

In [ ]:
%matplotlib inline

首先引用所有需要的包，其中pickle是用來讀取或是儲存二進位檔(我已經事先將Minist數據處理成二進位檔)。

In [ ]:
!apt-get install --no-install-recommends openmpi-bin libopenmpi-dev libopencv-dev python3-opencv python-opencv && ln -sf /usr/lib/x86_64-linux-gnu/libmpi_cxx.so /usr/lib/x86_64-linux-gnu/libmpi_cxx.so.1 && ln -sf /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so.12 && ln -sf /usr/lib/x86_64-linux-gnu/libmpi.so /usr/lib/x86_64-linux-gnu/libmpi.so.12 && pip install cntk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenmpi-dev is already the newest version (2.1.1-8).
openmpi-bin is already the newest version (2.1.1-8).
openmpi-bin set to manually installed.
libopencv-dev is already the newest version (3.2.0+dfsg-4ubuntu0.1).
The following NEW packages will be installed:
  python-opencv python3-opencv
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 1,069 kB of archives.
After this operation, 5,885 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-opencv amd64 3.2.0+dfsg-4ubuntu0.1 [535 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python3-opencv amd64 3.2.0+dfsg-4ubuntu0.1 [534 kB]
Fetched 1,069 kB in 2s (478 kB/s)
Selecting previously unselected package python-opencv.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../python-opencv_3

In [ ]:
from __future__ import print_function 
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import codecs
import math
import random
import pickle
from PIL import Image

import cntk as C
from cntk.ops import *
from cntk.layers import *
from cntk.losses import *
from cntk.metrics import *
from cntk.debugging import *
from cntk.logging import *
from cntk.learners import *
from cntk.train import *
from cntk.device import *

# 是否使用GPU
is_gpu = True
if is_gpu:
    try_set_default_device(gpu(0))
else:
    try_set_default_device(cpu())

/usr/local/lib/python3.6/dist-packages/cntk/cntk_py_init.py:56: UserWarning: Unsupported Linux distribution (ubuntu-18.04). CNTK supports Ubuntu 16.04 and above, only.
  warnings.warn('Unsupported Linux distribution (%s-%s). CNTK supports Ubuntu 16.04 and above, only.' % (__my_distro__, __my_distro_ver__))


ValueError: ignored

In [ ]:
C.__version__

'2.7'

將mnist數據進行轉換，請注意，在cntk需要將output轉為onehot(利用np.eye函數)。對於feature除以255，是為了將像素值控制在0~1之間，這樣收斂比較快

In [ ]:
train_data=None
test_data=None

with open('../Data/mnist_train.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('../Data/mnist_test.pkl', 'rb') as f:
    test_data = pickle.load(f)
    
def parse_mnist(data):
    features=[]
    labels=[]
    for row in data:
        labels.append(np.eye(10)[row[-1]].astype(np.float32))
        features.append(row[:-1].astype(np.float32)/255.0)#正規化
    return np.asarray(features),np.asarray(labels)



features,labels=parse_mnist(train_data)
print(features[:3])
print(features.shape)
print(labels.shape)


FileNotFoundError: ignored

In [ ]:
idxs=np.arange(0,train_data.shape[0])
random.shuffle(idxs)
idx=0
print(idxs.shape)
print(idxs)
def get_next_minibatch(minibatch_size):
    global idxs,idx
    features,labels=parse_mnist(train_data)
    x_features=[]
    y_labels=[]
    while len(x_features)<minibatch_size:
        x_features.append(features[idxs[idx]])
        y_labels.append(labels[idxs[idx]])
        idx+=1
        if idx>=len(idxs)-1:
            idx=0
            random.shuffle(idxs)
    return np.asarray(x_features).astype(np.float32),np.asarray(y_labels).astype(np.float32)

features_x,labels_y=get_next_minibatch(3)
print(features_x.shape)
print(labels_y.shape)

In [ ]:
features_x,labels_y=get_next_minibatch(32)
img=Image.fromarray(np.reshape(features_x[0,:]*255,(28,28)).astype(np.uint8))
plt.axis('off')
plt.imshow(img, cmap='gray', interpolation='nearest')

print(labels_y[0,:])
print(np.argmax(labels[0,:]))

以下我設計了三種autoencoder結構

In [ ]:
def autoencoder(x):#基本型
    x=Dense(784,activation=C.relu)(x)
    x=Dense(512,activation=C.relu)(x)
    x=Dense(256,activation=C.relu)(x)
    x=Dense(128,activation=C.relu)(x)
    x=Dense(64,activation=C.relu)(x)
    x=Dense(32,activation=None)(x)
    x=Dense(64,activation=C.relu)(x)
    x=Dense(128,activation=C.relu)(x)
    x=Dense(256,activation=C.relu)(x)
    x=Dense(512,activation=C.relu)(x)
    x=Dense(784,activation=C.relu)(x)
    x=clip(x,0,1)
    return x

In [ ]:
def autoencoder1(x):#加入批次正規化
    x=Dense(784,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(512,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(256,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(128,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(64,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(32,activation=None)(x)
    x=BatchNormalization()(x)
    x=Dense(64,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(128,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(256,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(512,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(784,activation=C.relu)(x)
    x=clip(x,0,1)
    return x

In [ ]:
def autoencoder2(x):#加入dropout
    x=Dense(784,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(512,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(256,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dropout(0.2)(x)
    x=Dense(128,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(64,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(32,activation=None)(x)
    x=Dense(64,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(128,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(256,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(512,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(784,activation=C.relu)(x)
    x=clip(x,0,1)
    return x

In [ ]:
input_var=C.input_variable(784, dtype=np.float32, name='input_var')

z=autoencoder1(input_var)

loss=reduce_mean(squared_error(z,input_var))
errs=sqrt(reduce_sum(squared_error(z,input_var)/reduce_sum(input_var)))

In [ ]:
progress_printer = ProgressPrinter(freq=200, tag='Training', num_epochs=300)

learning_rate=0.001
minibatch_size=64
num_epochs=3
learner = adam(z.parameters, lr=learning_rate_schedule([learning_rate], UnitType.sample, 300),
               momentum=momentum_as_time_constant_schedule([minibatch_size / -math.log(0.95)], epoch_size=300),
               l1_regularization_weight=0.0001, l2_regularization_weight=5e-3)

trainer = Trainer(z, (loss, errs), learner, progress_printer)

In [ ]:
for epoch_count in range(num_epochs):
    mbs = 0
    while mbs < len(features)/minibatch_size:
        features_x,labels_y=get_next_minibatch(minibatch_size)
        trainer.train_minibatch({input_var: features_x})
        if mbs%5==0:
            z.save('Models/autoencoder_cntk.onnx', format=C.ModelFormat.ONNX)
        mbs+=1
    trainer.summarize_training_progress()

問題
(1)為何最中間層的活化函數為None(還有哪些選擇呢?) tanh
(2)最後一層的活化函數你覺得relu,sigmoid,leaky relu哪效果比較好呢?
(3)三種自編碼器哪一種效果比較好(有沒有和預期不同的地方)
(4)如果數值正規化方式改為減127.5除以127.5，請問模型結構該如何調整呢?
(5)如果是你 ，還可以如何修改網路結構讓效果更好呢?

接下來我們比對一下輸入與輸出的差異

In [ ]:
features_x,labels_y=get_next_minibatch(minibatch_size)
results=z(features_x)#產生預測結果
#實際值
actual=np.reshape(features_x[0,:]*255,(28,28)).astype(np.uint8)
pred=np.reshape(results[0,:]*255,(28,28)).astype(np.uint8)
print(actual.shape)
print(pred.shape)
img=Image.fromarray(np.concatenate([actual,pred],axis=-1))
plt.axis('off')
plt.imshow(img, cmap='gray', interpolation='nearest')


# 去噪自編碼器 Denoise AutoEncoder

所謂的去噪自編碼器，輸入值是添加了噪音的數據，但是輸出卻希望他能還原回原來乾淨的數據，這個做法等於是強迫模型自己找出噪音與真實數據間的差異進而分離純化。

<img src="https://github.com/AllanYiin/DeepBelief_Course4_Examples/blob/master/Images/denoise.jpg?raw=1">

所以唯一需要修正之處在於產生添加噪音的輸入值。

In [ ]:
def get_next_noise_minibatch(minibatch_size):
    global idxs,idx
    features,labels=parse_mnist(train_data)
    x_features=[]
    x_noise=[]
    while len(x_features)<minibatch_size:
        x_features.append(features[idxs[idx]])
        x_noise.append(features[idxs[idx]]+np.random.standard_normal(784)*0.005)
        idx+=1
        if idx>=len(idxs):
            idx=0
            random.shuffle(idxs)
    return np.asarray(x_features).astype(np.float32),np.asarray(x_noise).astype(np.float32)

features_x,noise_x=get_next_noise_minibatch(3)
actual=np.reshape(features_x[0,:]*255,(28,28)).astype(np.uint8)
noise=np.reshape(noise_x[0,:]*255,(28,28)).astype(np.uint8)

img=Image.fromarray(np.concatenate([actual,noise],axis=-1))
plt.axis('off')
plt.imshow(img, cmap='gray', interpolation='nearest')

雖然輸入含噪音數據，但是損失函數則需要比對乾淨的原始值。

In [ ]:
input_var=C.input_variable(784, dtype=np.float32, name='input_var')
noise_var=C.input_variable(784, dtype=np.float32, name='input_var')

z_denoise=autoencoder1(noise_var)

loss=reduce_sum(squared_error(z_denoise,input_var))
errs=sqrt(reduce_sum(squared_error(z_denoise,input_var)/reduce_sum(input_var)))

In [ ]:
learning_rate=0.001
minibatch_size=64
num_epochs=3
progress_printer1 = ProgressPrinter(freq=200, tag='Training', num_epochs=300)

learner1 = adam(z_denoise.parameters, lr=learning_rate_schedule([learning_rate], UnitType.sample, 300),
               momentum=momentum_as_time_constant_schedule([minibatch_size / -math.log(0.95)], epoch_size=300),
               l1_regularization_weight=0.0001, l2_regularization_weight=5e-3)

trainer1 = Trainer(z_denoise, (loss, errs), learner1, progress_printer1)

for epoch_count in range(num_epochs):
    mbs = 0
    while mbs < len(features)/minibatch_size:
        raw_features,raw_noise=get_next_noise_minibatch(minibatch_size)
        trainer1.train_minibatch({input_var: raw_features,noise_var:raw_noise})
        if mbs%5==0:
            z_denoise.save('Models/denoise_autoencoder_cntk.onnx', format=C.ModelFormat.ONNX)
        mbs+=1
    #trainer1.summarize_training_progress()

比對原始數據、添加噪音數據以及最後模型生成值，可以看到神經網路學會了如何區分噪音與信息。

In [ ]:
raw_features,raw_noise=get_next_noise_minibatch(minibatch_size)
results=z_denoise(raw_noise)#產生預測結果
#實際值
actual=np.reshape(raw_features[0,:]*255,(28,28)).astype(np.uint8)
noise=np.reshape(raw_noise[0,:]*255,(28,28)).astype(np.uint8)
pred=np.reshape(results[0,:]*255,(28,28)).astype(np.uint8)

img=Image.fromarray(np.concatenate([actual,noise,pred],axis=-1))
plt.axis('off')
plt.imshow(img, cmap='gray', interpolation='nearest')

# 變分自編碼器（VAE）

變分自編碼器雖然名字叫做自編碼器，但實際上與自編碼器沒有太多關係，相對的，他反而很常被拿來與GAN相提並論，它的目的都是透過一個隱向量的分布來去生成一群樣本。變分自編碼器不管原始樣本的分布是如何，他都強制轉換為一個常態分配的分布，也就是他的作用是用來進行強制分布變換。

In [ ]:
def vae_encoder(x):
    x=Dense(784,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(512,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(256,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(128,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(64,activation=C.relu)(x)
    x=BatchNormalization()(x)
    return x

def vae_decoder(x):
    x=Dense(32,activation=None)(x)
    x=BatchNormalization()(x)
    x=Dense(64,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(128,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(256,activation=C.relu)(x)
    x=BatchNormalization()(x)
    x=Dense(512,activation=C.relu)(x)
 
 
    return x

latent_dim = 32 # 隱變量維度數

h = vae_encoder(input_var)

# 算p(Z|X)的均值和方差
z_mean = Dense(32)(h)
z_log_var = Dense(32)(h)


#epsilon表示隨機成分
epsilon =C.random.normal(shape=(latent_dim), mean=0.,scale=1)
z=z_mean +exp(z_log_var/2)#* epsilon

#如果不處理epsilon，則為AUTOENCODER
#如果處理epsilon，則為生成模型


#解碼層
decoder_mean = Dense(784, activation=sigmoid)
h_decoded = vae_decoder(z)
z_vae= decoder_mean(h_decoded)


In [ ]:
xent_loss = 784*binary_cross_entropy(z_vae,input_var)#重構損失
kl_loss = - 0.5 * reduce_sum(1 + z_log_var - square(z_mean) - exp(z_log_var), axis=-1)#KL散度
vae_loss = reduce_mean(xent_loss + kl_loss)

loss=vae_loss
errs=sqrt(reduce_sum(squared_error(z_vae,input_var)/reduce_sum(input_var)))

In [ ]:
progress_printer2 = ProgressPrinter(freq=200, tag='Training', num_epochs=300)

learning_rate=0.0001
minibatch_size=64
num_epochs=5
learner2 = adam(z_vae.parameters, lr=learning_rate_schedule([learning_rate], UnitType.sample, 300),
               momentum=momentum_as_time_constant_schedule([minibatch_size / -math.log(0.95)], epoch_size=300),
               l1_regularization_weight=0.0001, l2_regularization_weight=5e-3)

trainer2 = Trainer(z_vae, (loss, errs), learner2, progress_printer2)
for epoch_count in range(num_epochs):
    mbs = 0
    while mbs < len(features)/minibatch_size:
        raw_features,raw_labels=get_next_minibatch(minibatch_size)
        trainer2.train_minibatch({input_var: raw_features})
        if mbs%5==0:
            z_vae.save('Models/vae_cntk.model')
        mbs+=1
    trainer2.summarize_training_progress()

In [ ]:
raw_features,raw_labels=get_next_minibatch(minibatch_size)
results=z_vae(raw_features)#產生預測結果
#實際值
actual=np.reshape(raw_features[0,:]*255,(28,28)).astype(np.uint8)
pred=np.reshape(results[0,:]*255,(28,28)).astype(np.uint8)

img=Image.fromarray(np.concatenate([actual,pred],axis=-1))
plt.axis('off')
plt.imshow(img, cmap='gray', interpolation='nearest')